In [ ]:
# run `huggingface-cli login` on command line
# !pip install --upgrade transformers
# !pip install --upgrade tokenizers
# !pip install --upgrade torch

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
from accelerate import infer_auto_device_map, init_empty_weights
from transformers import AutoConfig, AutoTokenizer, AutoModelForCausalLM, LlamaTokenizer

checkpoint = 'meta-llama/Llama-2-7b-chat-hf'

config = AutoConfig.from_pretrained(checkpoint)#, use_auth_token=True)
with init_empty_weights():
    model = AutoModelForCausalLM.from_config(config)
model

In [ ]:
device_map = infer_auto_device_map(model, no_split_module_classes=["LlamaDecoderLayer"])
device_map

In [ ]:
tokenizer = LlamaTokenizer.from_pretrained(checkpoint)

model = AutoModelForCausalLM.from_pretrained(checkpoint, device_map=device_map)

model.hf_device_map

In [ ]:
prompt = 'Hello, tell me about yourself'

input_ids = tokenizer.encode(prompt, return_tensors='pt').to('cuda')

output = model.generate(input_ids, max_length=100, temperature=0.7, do_sample=True)

decoded_output = tokenizer.decode(output[0])
print(decoded_output)